In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import scipy 
import os
import unittest

In [ ]:
df_battery = pd.read_csv('../data/Battery_Dataset.csv')

In [ ]:
import numpy as np;
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
import csv
import pandas as pd
import sklearn as sk
from sklearn.model_selection import KFold 
from notebook.services.config import ConfigManager
cm = ConfigManager().update('notebook', {'limit_output': 1000})

import statistics as stc
import scipy.stats as stats

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
#One hot encoding (alphebetical order)
ohe = OneHotEncoder()
ACE = df_battery.loc[:,['anode','cathode','electrolyte']]
ACE = ohe.fit_transform(ACE)
ACE = pd.DataFrame(ACE.toarray())

ACE_col_name = ['A1','C1','C2','C3','E1','E2','E3']
for i in range(len(ACE.columns)):
    ACE = ACE.rename({ACE.columns[i]: ACE_col_name[i]}, axis=1) 
    
df_battery = pd.concat([ACE, df_battery],axis=1)

In [ ]:
data = df_battery

np.random.seed(66)
def data_split (data, test_ratio):
    
    total_row = df_battery.shape[0]
    test_row = round(total_row *test_ratio)
    train_row = total_row - test_row
    
    indices =np.random.permutation(total_row)
    train_indx, test_idx =indices[:train_row], indices[train_row:]
    train,test = df_battery.iloc[train_indx,:], df_battery.iloc[test_idx,:]

    X_test = test[['A1','C1','C2','C3','E1','E2','E3','Cycle','temperature','discharge_crate']]
#    y_test = test[['Charge_Capacity (Ah)']]
    y_test = test[['Discharge_Capacity (Ah)']]
#    y_test = test[['Charge_Energy (Wh)']]
#    y_test = test[['Discharge_Energy (Wh)']]
#    y_test = test[['Coulombic_Efficiency (%)']]
#    y_test = test[['Energy_Efficiency (%)']]
    
    X_train = train[['A1','C1','C2','C3','E1','E2','E3','Cycle','temperature','discharge_crate']]
#    y_train = train[['Charge_Capacity (Ah)']]
    y_train = train[['Discharge_Capacity (Ah)']]
#    y_train = train[['Charge_Energy (Wh)']]
#    y_train = train[['Discharge_Energy (Wh)']]
#    y_train = train[['Coulombic_Efficiency (%)']]
#    y_train = train[['Energy_Efficiency (%)']]
    return train,test,X_train,y_train,X_test,y_test
    
train,test,X_train,y_train,X_test,y_test = data_split(data,0.2)


In [ ]:
#k-fold cross validation
kf = KFold(n_splits=10,random_state =66,shuffle= True)
#print(X_test,y_test,X_train,y_train)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras import losses


def Deep_neural(lr, hidden_dimension, hidden_layers, reg=0):

    initializer = tf.keras.initializers.VarianceScaling(scale=2)
    
    regularizer = tf.keras.regularizers.l2(reg)

    model = Sequential()
    model.add(Dense(hidden_dimension, activation='relu', kernel_initializer=initializer,
                           kernel_regularizer=regularizer, input_dim=20))
    for i in range(hidden_layers):

            model.add(Dense(hidden_dimension, activation='relu', kernel_initializer=initializer,
                           kernel_regularizer=regularizer))
    
    model.add(Dense(1,activation ='linear',kernel_initializer=initializer,
                           kernel_regularizer =regularizer))

    
    model.compile(metrics='mse',optimizer = optimizers.Adam(learning_rate =lr),loss=losses.MeanAbsolutePercentageError())
    return model

In [ ]:
#Hyperparameter tuning
learning_rate = [0.001,0.01, 0.1]
hidden_dim = np.arange(5,21,1).tolist()
hidden_l = np.arange(5,21,1).tolist()

Results_name = ["learning_rate", "hidden_dimension", "hidden_layeres", "RMSE"]
Results =[]



for i in learning_rate:
    for d in hidden_dim:
        for l in hidden_l:
            for train_index, test_index in kf.split(X_train):
                print("Train:", train_index, "Validation:", test_index)
                X_training, X_validate = X_train.iloc[train_index], X_train.iloc[test_index]
                y_training, y_validate = y_train.iloc[train_index], y_train.iloc[test_index]
                initial_model = Deep_neural(lr=i, hidden_dimension=d, hidden_layers=l)
                history = initial_model.fit(x=np.asarray(X_training), y=np.asarray(y_training), batch_size=15, epochs=1)
                A = initial_model.evaluate(x=np.asarray(X_validate), y=np.asarray(y_validate)[1]
                rmse = math.sqrt(A)
                Results.append([i, d, l, rmse])                

In [ ]:
#Train error
for train_index, test_index in kf.split(X_train):
                #print("Train:", train_index, "Validation:", test_index)
                X_training, X_validate = X_train.iloc[train_index], X_train.iloc[test_index]
                y_training, y_validate = y_train.iloc[train_index], y_train.iloc[test_index]
                
                initial_model = Deep_neural(
                lr=0.001, hidden_dimension=20, hidden_layers=8)
                np.random.seed(66)
                history = initial_model.fit(x=X_training, y=y_training, batch_size=10, epochs=1000,verbose=0)
                np.random.seed(66)
                y_predict = initial_model.predict(X_validate)
                

In [ ]:
Predicted_train_value =[]

for i in Predicted:
    for b in i:
        for c in b:
            Predicted_train_value.append(c)

In [ ]:
Predicted_train_values = pd.DataFrame(Predicted_train_value)
Experimental_train_values = pd.DataFrame(Experimental)

In [ ]:
Experimental_train_value =[]

for i in range(Experimental_train_values.shape[0]):
    Ex=Experimental_train_values.iloc[i,0].to_string(index=False)
    Ex_num = Ex.split("\n ")
    a = np.array([float(i) for i in Ex_num])
    Experimental_train_value.extend(a)